In [1]:
import pandas as pd
pd.set_option('display.max_columns', 500)
from datetime import datetime
import numpy as np
import timeit
import json

In [2]:
start_time = timeit.default_timer()


df_2019_may = pd.read_json('json_datasets/Kickstarter_2019-05-16T03_20_20_822Z.json', lines=True)


elapsed = timeit.default_timer() - start_time
print(f'time: {elapsed}')

time: 144.138165386


In [3]:
# Save the creation datetime for calculating age of project later
df_2019_may_collected_on = datetime(2019, 5, 16)

In [4]:
# clean up the format and make a deep copy
# start_time = timeit.default_timer()


df_2019_may = df_2019_may['data'].apply(pd.Series)
# df_2019_may_copy = df_2019_may.copy(deep=True)
print(f'Total_rows: {len(df_2019_may)}')


# elapsed = timeit.default_timer() - start_time
# print(f'time: {elapsed}')

df_2019_may.head(3)

Total_rows: 210857


,id,photo,name,blurb,goal,pledged,state,slug,disable_communication,country,currency,currency_symbol,currency_trailing_code,deadline,state_changed_at,created_at,launched_at,staff_pick,is_starrable,backers_count,static_usd_rate,usd_pledged,converted_pledged_amount,fx_rate,current_currency,usd_type,creator,location,category,profile,spotlight,urls,source_url,friends,is_starred,is_backing,permissions
0,177028447,{'key': 'assets/011/281/287/db432df50103184abf...,The Polk Street Mural Project (Canceled),I am painting a Mural in San Francisco on Polk...,5000.0,45.0,canceled,the-polk-street-mural-project,False,US,USD,$,True,1309748392,1304547294,1301443012,1301972392,False,False,3,1.0,45.0,45,1.0,USD,domestic,"{'id': 483566317, 'name': 'Sirron Norris', 'is...","{'id': 2487956, 'name': 'San Francisco', 'slug...","{'id': 24, 'name': 'Performance Art', 'slug': ...","{'id': 28620, 'project_id': 28620, 'state': 'i...",False,{'web': {'project': 'https://www.kickstarter.c...,https://www.kickstarter.com/discover/categorie...,NaN,NaN,NaN,NaN
1,407610332,{'key': 'assets/012/953/120/60f7c7bed3211630d2...,Voilà | Specialty Instant Coffee | World's Bes...,"Specialty instant coffee from top US roasters,...",26000.0,27121.0,successful,voila-specialty-instant-coffee-worlds-best-ins...,False,US,USD,$,True,1472828209,1472828209,1467652500,1470236209,True,False,274,1.0,27121.0,27121,1.0,USD,domestic,"{'id': 993748579, 'name': 'Kent Sheridan', 'is...","{'id': 2362495, 'name': 'Bend', 'slug': 'bend-...","{'id': 307, 'name': 'Drinks', 'slug': 'food/dr...","{'id': 2586539, 'project_id': 2586539, 'state'...",True,{'web': {'project': 'https://www.kickstarter.c...,https://www.kickstarter.com/discover/categorie...,NaN,NaN,NaN,NaN
2,1187682690,{'key': 'assets/024/180/051/eab3c1b537669b76d2...,Dead Bird brewing Co,"Milwaukee’s newest craft brewery, focusing on ...",10000.0,10636.0,successful,dead-bird-brewing-co,False,US,USD,$,True,1553985061,1553985061,1538730098,1550100661,False,False,82,1.0,10636.0,10636,1.0,USD,domestic,"{'id': 1739366281, 'name': 'Nick Kocis', 'is_r...","{'id': 2451822, 'name': 'Milwaukee', 'slug': '...","{'id': 307, 'name': 'Drinks', 'slug': 'food/dr...","{'id': 3485013, 'project_id': 3485013, 'state'...",True,{'web': {'project': 'https://www.kickstarter.c...,https://www.kickstarter.com/discover/categorie...,NaN,NaN,NaN,NaN


In [5]:
# # Check if data has same features
# df_2019_may_cols = df_2019_may.columns
# columns_original  = ['id', 'photo', 'name', 'blurb', 'goal', 'pledged', 'state', 'slug', 'disable_communication', 'country', 'country_displayable_name', 'currency', 'currency_symbol', 'currency_trailing_code', 'deadline', 'state_changed_at', 'created_at', 'launched_at', 'staff_pick', 'is_starrable', 'backers_count', 'static_usd_rate', 'usd_pledged', 'converted_pledged_amount', 'fx_rate', 'current_currency', 'usd_type', 'creator', 'location', 'category', 'profile', 'spotlight', 'urls', 'source_url', 'friends', 'is_starred', 'is_backing', 'permissions']
# diff = list(set(df_2019_may_cols) - set(columns_original))
# diff2 = list(set(columns_original) - set(df_2019_may_cols))
# print(diff)
# print(diff2)

In [6]:
# # inspect canceled and suspended states
# df_2019_may.state.value_counts()

In [7]:
# drop canceled and suspended states
df_2019_may.drop((df_2019_may[((df_2019_may.state == 'suspended') | 
                                         (df_2019_may.state == 'canceled'))].index), inplace=True)

# check for duplicates
print(f"Rows with duplicate project_id: {df_2019_may.duplicated(['id']).sum()}")

# drop duplicates
df_2019_may.drop_duplicates(['id'], inplace=True)
print(f'Total_rows after dropping duplicates and suspended/canceled states: {len(df_2019_may)}')


Rows with duplicate project_id: 27148
Total_rows after dropping duplicates and suspended/canceled states: 174510


In [8]:
# # get links to project and creator pages
# start_time = timeit.default_timer()


# df_2019_may['project_link'] = 'no_val'
# project_link_loc = df_2019_may.columns.get_loc('project_link')
# for n in range(len(df_2019_may)):
#     project_link = df_2019_may.iloc[n].urls['web']['project']
#     df_2019_may.iat[n, project_link_loc] = project_link
    
# df_2019_may['creator_link'] = 'no_val'
# creator_link_loc = df_2019_may.columns.get_loc('creator_link')
# for n in range(len(df_2019_may)):
#     creator_link = df_2019_may.iloc[n].creator['urls']['web']['user']
#     df_2019_may.iat[n, creator_link_loc] = creator_link  
    
    
# elapsed = timeit.default_timer() - start_time
# print(f'time: {elapsed}')

In [9]:
# # get main_category id
# start_time = timeit.default_timer()


# categories_map = {1:'Art',3:'Comics', 17:'Theater', 15:'Photography', 
#                 7:'Design', 6: 'Dance', 11:'Film & Video', 12:'Games', 16:'Technology', 
#                 13:'Journalism', 18:'Publishing', 9:'Fashion', 10:'Food', 14:'Music', 26:'Crafts'}
# categories_df = pd.DataFrame(categories_map, index=[0])


# df_2019_may['main_category'] = 'no_value'
# cat_loc = df_2019_may.columns.get_loc('main_category')


# for n in (range(len(df_2019_may))):
#     if 'parent_id' not in df_2019_may.iloc[n]['category'].keys():
#         cat_id = df_2019_may.iloc[n]['category']['id']
#         df_2019_may.iat[n, cat_loc] = cat_id
#     elif pd.isnull(df_2019_may.iloc[n]['category']['parent_id']):
#         cat_id = df_2019_may.iloc[n]['category']['id']
#         df_2019_may.iat[n, cat_loc] = cat_id
#     else:
#         par_id = df_2019_may.iloc[n]['category']['parent_id']
#         df_2019_may.iat[n, cat_loc] = par_id
        
# df_2019_may['main_category'] = df_2019_may['main_category'].apply(lambda x: categories_df[x])


# elapsed = timeit.default_timer() - start_time
# print(f'time: {elapsed}')

# df_2019_may.head(1)

In [10]:
# # keep these features
# df_main_feats = ['id', 'name', 'blurb', 'goal', 'pledged', 'state', 'country',
# 'currency', 'deadline', 'launched_at', 'staff_pick', 'main_category',
# 'backers_count', 'static_usd_rate', 'usd_pledged' ]
# # Drop these 
# df_drop_feats = list(set(df_2019_may_cols) - set(df_main_feats))
# print(f'Columns removed: {df_drop_feats}')

# # drop features
# df_2019_may.drop(df_drop_feats, axis=1, inplace=True)

In [11]:
# convert unix dates to datetime objects
df_2019_may.deadline = df_2019_may.deadline.apply(lambda x: (datetime.fromtimestamp(x)))
df_2019_may.launched_at = df_2019_may.launched_at.apply(lambda x: (datetime.fromtimestamp(x)))

In [12]:
# rename columns
df_2019_may.rename(columns={'id':'project_id'}, inplace=True)
df_2019_may.rename(columns={'name':'project_name'}, inplace=True)
df_2019_may.rename(columns={'blurb':'project_description'}, inplace=True)
df_2019_may.rename(columns={'state':'outcome_state'}, inplace=True)


In [13]:
# add project duration feature
df_2019_may['duration'] = df_2019_may.deadline - df_2019_may.launched_at
df_2019_may['days_from_launch'] =(df_2019_may_collected_on - df_2019_may.launched_at).apply(lambda x: x.days)

# # add feature 'in_mid_duration_range' indicating whether project is about halfway through
# df_2019_may['duration_mid_point'] = df_2019_may.duration.apply(lambda x: x.days) / 2
# df_2019_may['lower_bound_mid_point_range'] = df_2019_may.duration_mid_point.apply(lambda x: x - 5)
# df_2019_may['upper_bound_mid_point_range'] = df_2019_may.duration_mid_point.apply(lambda x: x + 5)
# df_2019_may['in_mid_duration_range'] = df_2019_may.days_from_launch.between(df_2019_may.lower_bound_mid_point_range, df_2019_may.upper_bound_mid_point_range)

# df_2019_may.drop(['duration_mid_point', 'lower_bound_mid_point_range', 'upper_bound_mid_point_range'], axis=1, inplace=True)


In [14]:
# df_2019_may['duration'][0].quantile(.1)
df_2019_may['duration_10_percent'] = df_2019_may.duration.apply(lambda x: x.days / 10 ) 


In [15]:
# add feature 'in_mid_duration_range' indicating whether project is about halfway through
df_2019_may['lower_bound'] = df_2019_may.duration_10_percent.apply(lambda x: x if x < 5 else x - 5)
df_2019_may['upper_bound'] = df_2019_may.duration_10_percent.apply(lambda x: x + 5)
df_2019_may['in_range'] = df_2019_may.days_from_launch.between(df_2019_may.lower_bound, df_2019_may.upper_bound)

# df_2019_may.drop(['duration_mid_point', 'lower_bound_mid_point_range', 'upper_bound_mid_point_range'], axis=1, inplace=True)

In [16]:
# calculate usd_goal
df_2019_may['usd_goal'] = df_2019_may.goal * df_2019_may.static_usd_rate


In [17]:
# Replace goal and rearrange
# col_list = df_2019_may.columns.to_list()
# print(col_list)
col_list_rearrange = ['project_id', 'project_name', 'project_description', 'outcome_state', 'in_range',
                      'duration_10_percent', 'days_from_launch', 
                      'usd_goal', 'usd_pledged', 'currency', 'launched_at', 'deadline', 'staff_pick', 'country', 
                      'duration' , 'backers_count']
df_2019_may = df_2019_may[col_list_rearrange]
df_2019_may.head(1)
                      

,project_id,project_name,project_description,outcome_state,in_range,duration_10_percent,days_from_launch,usd_goal,usd_pledged,currency,launched_at,deadline,staff_pick,country,duration,backers_count
1,407610332,Voilà | Specialty Instant Coffee | World's Bes...,"Specialty instant coffee from top US roasters,...",successful,False,3.0,1015,26000.0,27121.0,USD,2016-08-03 10:56:49,2016-09-02 10:56:49,True,US,30 days,274


In [18]:
df_2019_may_copy = df_2019_may.copy(deep=True)
# convert datetime columns before exporting
# df_2019_may.duration = df_2019_may.duration.apply(lambda x: x.days) 
df_2019_may.launched_at = df_2019_may.launched_at.apply(lambda x: x.strftime('%m-%d-%Y'))
df_2019_may.deadline = df_2019_may.deadline.apply(lambda x: x.strftime('%m-%d-%Y'))

In [19]:
# export cleaned dataframe to csv
df_2019_may.to_csv('clean_datasets_10/main_df_2019_may.csv', index=False)